In [15]:
import openai
import ipywidgets as widgets
from IPython.display import display

# Inicializar la clave API como None
api_key = None

# Función para sugerir títulos
def sugerir_titulos(texto):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Genera 5 sugerencias de títulos optimizados en SEO, para este texto: {texto}"}
        ]
    )
    titulos = response.choices[0]['message']['content'].split('\n')
    return [titulo.strip() for titulo in titulos if titulo.strip()]

# Función para sugerir keywords
def sugerir_keywords(texto):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Genera keywords relevantes segun la tendencia de googleTrends para este texto: {texto}"}
        ]
    )
    keywords = response.choices[0]['message']['content']
    return keywords.split(", ")

# Función para generar un resumen breve (menor a 1000 caracteres)
def generar_resumen(texto):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Resume el siguiente texto en menos de 1000 caracteres: {texto}"}
        ]
    )
    resumen = response.choices[0]['message']['content']
    return resumen[:1000]  # Asegurar que el resumen no exceda los 1000 caracteres

# Función para generar imagen usando DALL-E
def generar_imagen(resumen):
    response = openai.Image.create(
        prompt=resumen,
        n=1,
        size="512x512"  # Tamaño de la imagen
    )
    return response['data'][0]['url']

# Función para generar bajada
def generar_bajada(texto):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Genera una bajada breve en 150 palabras, para este texto: {texto}"}
        ]
    )
    return response.choices[0]['message']['content']

# Función para generar información extra
def generar_informacion_extra(texto):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Proporciona información adicional que sea interesante para este texto: {texto}"}
        ]
    )
    return response.choices[0]['message']['content']

# Función para revisar gramática
def revisar_gramatica(texto):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Revisa la gramática de este texto: {texto}"}
        ]
    )
    return response.choices[0]['message']['content']

# Función para manejar el evento de botón
def procesar_texto(b):
    global api_key
    if api_key is None:
        output_area.append_stdout("Por favor, ingresa tu API key de OpenAI.\n")
        return

    openai.api_key = api_key  # Establecer la clave API
    
    texto = texto_input.value
    if not texto.strip():
        output_area.append_stdout("Por favor, ingresa un texto.\n")
        return
    
    opcion = opcion_selector.value
    
    if opcion == 1:
        titulos = sugerir_titulos(texto)
        output_area.append_stdout("\nSugerencias de Títulos:\n")
        for titulo in titulos:
            output_area.append_stdout(titulo + "\n")
    
    elif opcion == 2:
        resumen = generar_resumen(texto)
        output_area.append_stdout(f"\nResumen para la imagen: {resumen}\n")
        imagen_url = generar_imagen(resumen)
        output_area.append_stdout(f"\nURL de la imagen generada: {imagen_url}\n")
    
    elif opcion == 3:
        keywords = sugerir_keywords(texto)
        output_area.append_stdout("\nSugerencias de Keywords:\n")
        output_area.append_stdout(", ".join(keywords) + "\n")
    
    elif opcion == 4:
        bajada = generar_bajada(texto)
        output_area.append_stdout(f"\nBajada (Resumen): {bajada}\n")
    
    elif opcion == 5:
        info_extra = generar_informacion_extra(texto)
        output_area.append_stdout(f"\nInformación Extra: {info_extra}\n")
    
    elif opcion == 6:
        gramatica = revisar_gramatica(texto)
        output_area.append_stdout(f"\nRevisión Gramatical:\n{gramatica}\n")

# Función para manejar el evento de ingreso de API key
def set_api_key(b):
    global api_key
    api_key = api_key_input.value.strip()  # Guardar la API key
    output_area.append_stdout("API key establecida. Puedes procesar el texto ahora.\n")

# Widgets para la interfaz gráfica
api_key_input = widgets.Password(  # Cambiado a Password
    placeholder='Ingresa tu API key aquí...',
    description='API Key:',
    layout=widgets.Layout(width='90%')
)

texto_input = widgets.Textarea(
    placeholder='Ingresa tu texto aquí...',
    description='Texto:',
    layout=widgets.Layout(width='90%', height='150px')
)

opcion_selector = widgets.Dropdown(
    options=[
        ('Sugerir Títulos', 1),
        ('Generar Imagen', 2),
        ('Sugerir Keywords', 3),
        ('Generar Bajada', 4),
        ('Generar Información Extra', 5),
        ('Revisar Gramática', 6),
    ],
    description='Opciones:',
)

boton_set_api_key = widgets.Button(description="Establecer API Key")
boton_procesar = widgets.Button(description="Procesar")
output_area = widgets.Output()

# Conectar los botones a las funciones
boton_set_api_key.on_click(set_api_key)
boton_procesar.on_click(procesar_texto)

# Mostrar los widgets
display(api_key_input, boton_set_api_key, texto_input, opcion_selector, boton_procesar, output_area)


Password(description='API Key:', layout=Layout(width='90%'), placeholder='Ingresa tu API key aquí...')

Button(description='Establecer API Key', style=ButtonStyle())

Textarea(value='', description='Texto:', layout=Layout(height='150px', width='90%'), placeholder='Ingresa tu t…

Dropdown(description='Opciones:', options=(('Sugerir Títulos', 1), ('Generar Imagen', 2), ('Sugerir Keywords',…

Button(description='Procesar', style=ButtonStyle())

Output()